In [1]:
pip install chromedriver-autoinstaller


C:\Users\User\Web_Crawling>doskey defects4j=perl C:\Users\User\defects4j\framework\bin\defects4j $* 
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install selenium


C:\Users\User\Web_Crawling>doskey defects4j=perl C:\Users\User\defects4j\framework\bin\defects4j $* 
Note: you may need to restart the kernel to use updated packages.


In [4]:
import chromedriver_autoinstaller
from selenium import webdriver
import time

class Driver:
    @staticmethod
    def driver_init():
        chromedriver_autoinstaller.install()
        options = webdriver.ChromeOptions()
#         options.add_argument('headless') # jupyter notebook 환경에서 주석처리
        options.add_argument('lang=ko_KR')
        options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36")
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-notifications')
        options.add_argument('--disable-popup-blocking')
        options.add_argument('--disable-dev-shm-usage')
        options.add_argument('--disable-browser-side-navigation')
        options.add_argument('disable-gpu')

        driver = webdriver.Chrome(options=options)
        driver.set_window_size(1600, 1100)

        return driver

In [5]:
driver = Driver.driver_init()

In [6]:
urls = [
        ('https://frizm.co.kr/product/list.html?cate_no=24',
         'top', 17), #아우터
        ('https://frizm.co.kr/product/list.html?cate_no=52',
         'top', 37), #상의
        ('https://frizm.co.kr/product/list.html?cate_no=26',
         'bottom', 14), #하의
         ]

product_paths = []

## 아우터/상의/하의 분류 루프
for (url,cat, pages_cnt) in urls:
    ## 각 옷 분류의 페이지 갯수 루프
    for page_num in range(pages_cnt):
        try:
            driver.get(url + "&page=" + str(page_num))
        except:
            continue

        products = driver.find_elements('xpath', '//ul[@class="prdList grid4"]/li')
        products_cnt = len(products)

        ## 각 product의 path를 product_paths 리스트에 저장
        for i in range(products_cnt):
            product = driver.find_elements('xpath', '//ul[@class="prdList grid4"]/li')[i]
            product_path = product.find_element('xpath', './div[@class="thumbnail"]/a').get_attribute('href')
            product_paths.append(product_path)

print(len(product_paths))

2720


In [25]:
def crawl_pages(product_paths):
    
    base_dict= {"baseCat":[], "baseName": [], "baseCode": [], "baseUrl": [], "colorName": [],
            "baseGender": [], "price": [], "basePrice": [], "colorFabric": [], "imageUrl": [],
            "size": [], "isAvailable": []
            }
    
    ## 드라이버 초기화
    driver = Driver.driver_init()
    ## 각 product에 대한 정보 추출
    for product_path in product_paths:
        driver.get(product_path)

        cat = 'top'
        if "category/24" in product_path or "category/52" in product_path:
            cat = 'top'
        elif "category/26" in product_path:
            cat = 'bottom'

        baseURL = product_path
        base_gender = 0
        text_info = driver.find_element('xpath', '//div[@class="infoArea"]')
        imageURL = driver.find_element('xpath','//img[@class="BigImage "]').get_attribute('src')

        base_name = text_info.find_element('xpath', './div[@class="headingArea"]/h2').text
        base_code = ""
        color_name = ""
        ## color가 명시되지 않은 옷은 제외(패키지 상품)
        if "_" in base_name:
            base_code = base_name.split("_")[0]
            ## base_code 앞에 협업 브랜드 텍스트 제거
            if "] " in base_code:
                base_code = base_code.split("] ")[1]

            ## color_name 뒤에 협업 브랜드 텍스트 제거
            color_name = base_name.split("_")[1]
            if "[" in color_name:
                color_name = color_name.split("[")[0]
        else:
            continue

        base_price = ""
        price = ""
        color_fabric = ""
        trs = text_info.find_elements("xpath",'./div[2]/table/tbody/tr') #body의 table 항목 갯수                       
        if len(trs) == 5:
            base_price = text_info.find_element("xpath",'.//strong[@id="span_product_price_text"]').text
            price = text_info.find_element("xpath",'.//span[@id="span_product_price_sale"]').text
            color_fabric = text_info.find_element("xpath",'./div[2]/table/tbody/tr[5]/td/span').text
        elif len(trs) == 4:
            base_price = text_info.find_element("xpath",'.//strong[@id="span_product_price_text"]').text
            price = base_price
            color_fabric = text_info.find_element("xpath",'./div[2]/table/tbody/tr[4]/td/span').text
        elif len(trs) == 3:
            base_price = text_info.find_element("xpath",'.//strong[@id="span_product_price_text"]').text
            price = base_price
            color_fabric = ""
        elif len(trs) == 2:
            base_price = ""
            price = ""
            color_fabric = ""
            

        size_elements = text_info.find_elements("xpath",'.//optgroup[@label="사이즈"]/option')
        size = "M"
        is_available = True

        for element in size_elements:
            size = element.text.replace(" [품절]","")
            if "[품절]" in element.text:
                is_available = False
            else:
                is_available = True

            base_dict["baseCat"].append(cat)
            base_dict["baseName"].append(base_name)
            base_dict["baseCode"].append(base_code)
            base_dict["baseUrl"].append(baseURL)
            base_dict["colorName"].append(color_name)
            base_dict["baseGender"].append(0)
            base_dict["price"].append(price)
            base_dict["basePrice"].append(base_price)
            base_dict["colorFabric"].append(color_fabric)
            base_dict["imageUrl"].append(imageURL)
            base_dict["size"].append(size)
            base_dict["isAvailable"].append(is_available)
            
    return base_dict

In [26]:
import pandas as pd

In [27]:
dict1 = crawl_pages(product_paths[:500])
df1 = pd.DataFrame.from_dict(dict1)

print(df1)

     baseCat                              baseName  \
0        top         Paulo drizzler jacket _ beige   
1        top         Paulo drizzler jacket _ beige   
2        top         Paulo drizzler jacket _ beige   
3        top         Paulo drizzler jacket _ black   
4        top         Paulo drizzler jacket _ black   
...      ...                                   ...   
1492     top       Durable N-1 Deck jacket _ black   
1493     top       Durable N-1 Deck jacket _ black   
1494     top  Gingham check blouson jacket _ black   
1495     top  Gingham check blouson jacket _ black   
1496     top  Gingham check blouson jacket _ black   

                           baseCode  \
0            Paulo drizzler jacket    
1            Paulo drizzler jacket    
2            Paulo drizzler jacket    
3            Paulo drizzler jacket    
4            Paulo drizzler jacket    
...                             ...   
1492       Durable N-1 Deck jacket    
1493       Durable N-1 Deck jacket    


In [28]:
dict2 = crawl_pages(product_paths[500:1000])
df2 = pd.DataFrame.from_dict(dict2)

print(df2)

     baseCat                               baseName  \
0        top           William hunting coat _ olive   
1        top           William hunting coat _ olive   
2        top           William hunting coat _ olive   
3        top           William hunting coat _ black   
4        top           William hunting coat _ black   
...      ...                                    ...   
1239     top   OG Heavyweight pullover hoody _ gray   
1240     top   OG Heavyweight pullover hoody _ gray   
1241     top  OG Heavyweight pullover hoody _ taupe   
1242     top  OG Heavyweight pullover hoody _ taupe   
1243     top  OG Heavyweight pullover hoody _ taupe   

                            baseCode  \
0              William hunting coat    
1              William hunting coat    
2              William hunting coat    
3              William hunting coat    
4              William hunting coat    
...                              ...   
1239  OG Heavyweight pullover hoody    
1240  OG Heavyweigh

In [29]:
dict3 = crawl_pages(product_paths[1000:1500])
df3 = pd.DataFrame.from_dict(dict3)

print(df3)

     baseCat                                           baseName  \
0        top               OG Heavyweight pullover hoody _ navy   
1        top               OG Heavyweight pullover hoody _ navy   
2        top               OG Heavyweight pullover hoody _ navy   
3        top              OG Heavyweight pullover hoody _ black   
4        top              OG Heavyweight pullover hoody _ black   
...      ...                                                ...   
1490     top  [PENFIELD X FRIZMWORKS] Hawaiian logo tee _ lemon   
1491     top  [PENFIELD X FRIZMWORKS] Hawaiian logo tee _ lemon   
1492     top  [PENFIELD X FRIZMWORKS] Hawaiian logo tee _ black   
1493     top  [PENFIELD X FRIZMWORKS] Hawaiian logo tee _ black   
1494     top  [PENFIELD X FRIZMWORKS] Hawaiian logo tee _ black   

                            baseCode  \
0     OG Heavyweight pullover hoody    
1     OG Heavyweight pullover hoody    
2     OG Heavyweight pullover hoody    
3     OG Heavyweight pullover hoody

In [30]:
dict4 = crawl_pages(product_paths[1500:2000])
df4 = pd.DataFrame.from_dict(dict4)

print(df4)

     baseCat                                           baseName  \
0        top  [PENFIELD X FRIZMWORKS] P&F Paisley logo tee _...   
1        top  [PENFIELD X FRIZMWORKS] P&F Paisley logo tee _...   
2        top  [PENFIELD X FRIZMWORKS] P&F Paisley logo tee _...   
3        top  [PENFIELD X FRIZMWORKS] P&F Paisley logo tee _...   
4        top  [PENFIELD X FRIZMWORKS] P&F Paisley logo tee _...   
...      ...                                                ...   
1100     top                 FZ Pingpong club sweatshirt _ navy   
1101     top                 FZ Pingpong club sweatshirt _ navy   
1102     top                 FZ Pingpong club sweatshirt _ gray   
1103     top                 FZ Pingpong club sweatshirt _ gray   
1104     top                 FZ Pingpong club sweatshirt _ gray   

                          baseCode  \
0            P&F Paisley logo tee    
1            P&F Paisley logo tee    
2            P&F Paisley logo tee    
3            P&F Paisley logo tee    
4    

In [32]:
dict5 = crawl_pages(product_paths[2000:2500])
df5 = pd.DataFrame.from_dict(dict5)

print(df5)

     baseCat                           baseName                    baseCode  \
0        top           OG Cashmere knit _ ivory           OG Cashmere knit    
1        top           OG Cashmere knit _ ivory           OG Cashmere knit    
2        top           OG Cashmere knit _ ivory           OG Cashmere knit    
3        top         OG Cashmere knit _ oatmeal           OG Cashmere knit    
4        top         OG Cashmere knit _ oatmeal           OG Cashmere knit    
...      ...                                ...                         ...   
1045  bottom  Nylon ripstop sweat pants _ black  Nylon ripstop sweat pants    
1046  bottom  Nylon ripstop sweat pants _ black  Nylon ripstop sweat pants    
1047  bottom         Piping sweat pants _ ivory         Piping sweat pants    
1048  bottom         Piping sweat pants _ ivory         Piping sweat pants    
1049  bottom         Piping sweat pants _ ivory         Piping sweat pants    

                                                bas

In [33]:
dict6 = crawl_pages(product_paths[2500:])
df6 = pd.DataFrame.from_dict(dict6)

print(df6)

    baseCat                           baseName                 baseCode  \
0    bottom     Piping sweat pants _ ash khaki      Piping sweat pants    
1    bottom     Piping sweat pants _ ash khaki      Piping sweat pants    
2    bottom     Piping sweat pants _ ash khaki      Piping sweat pants    
3    bottom  Piping sweat pants _ melange gray      Piping sweat pants    
4    bottom  Piping sweat pants _ melange gray      Piping sweat pants    
..      ...                                ...                      ...   
359  bottom     OG Wool regular slacks _ black  OG Wool regular slacks    
360  bottom     OG Wool regular slacks _ black  OG Wool regular slacks    
361  bottom             OG Wide slacks _ beige          OG Wide slacks    
362  bottom             OG Wide slacks _ beige          OG Wide slacks    
363  bottom             OG Wide slacks _ beige          OG Wide slacks    

                                               baseUrl      colorName  \
0    https://frizm.co.kr/p

In [34]:
df_final = pd.concat([df1, df2, df3, df4, df5, df6])

In [38]:
df_final.to_csv('out.csv',index=False)